In [1]:
import collections
from collections import defaultdict
from sklearn import svm
from sklearn import cross_validation
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import metrics
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import SelectPercentile, f_classif, chi2
from sklearn.pipeline import Pipeline
import numpy as np
import pickle
import json

In [39]:
# specify dir
Dir = 'D:/Depression_project/data/reddit'

# Load data

In [40]:
text_control = []
with open(Dir+'/'+'randomSubreddits2017_selected.txt',"r") as f:
    data = f.readlines()
text_control = [x.strip() for x in data] 
len(text_control)

98339

In [4]:
print (text_control[:2])
text_control[0].split()

['want see how would address what predat eat food none charact eat meat entir movi be slightli bother bojack horseman do fantast dark episod show differ chicken eat chicken that be function member societi', 'how tall be']


['want',
 'see',
 'how',
 'would',
 'address',
 'what',
 'predat',
 'eat',
 'food',
 'none',
 'charact',
 'eat',
 'meat',
 'entir',
 'movi',
 'be',
 'slightli',
 'bother',
 'bojack',
 'horseman',
 'do',
 'fantast',
 'dark',
 'episod',
 'show',
 'differ',
 'chicken',
 'eat',
 'chicken',
 'that',
 'be',
 'function',
 'member',
 'societi']

In [41]:
# read data
text_depr = []
with open(Dir+'/'+'mentalSubreddits2017_selected.txt',"r") as f:
    data = f.readlines()
text_depr = [x.strip() for x in data] 
print (len(text_depr))
text_depr[:3]

98745


['anybodi els look find reason live can not find',
 'bad part be have live crazi fuck world',
 'what do everyon do their free time summer break colleg blue seemingli my friend can not stand long summer break colleg bit month leav lot free time even work long shift week work anybodi els find case be say anybodi have idea how can fill my free time there be onli so much exercis netflix cook person can do haha']

# Clean data

In [2]:
def get_words(docs):
    words = []
    for doc in docs:
        for word in doc.split():
            words.append(word)
    print ("The number of unique words: "+ str(len(set((words)))))
    return words

# define a funciton to find words occurring less than n times and delete that word
def findWordLessThan(counter, n):
    uncommon = []
    for k, v, in counter.items():
        if v < n:
            uncommon.append(k)            
    print (len(uncommon))
    return uncommon

def findWordMoreThan(counter, n):
    common = []
    for k, v, in counter.items():
        if v > n:
            common.append(k)
    print (len(common))
    return common

In [42]:
text_control = text_control[:50000]
text_depr = text_depr[:50000]

## Clean control posts

In [43]:
words_control = get_words(text_control)
count = collections.Counter(words_control)

The number of unique words: 46097


In [44]:
uncommon = findWordLessThan(count, 11)
common = findWordMoreThan(count, 700)

39966
213


In [25]:
"""""
# check
for k, v in count.items():
    if k in common:
        print (k, v)

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-25-373bc34c067d>, line 4)

In [45]:
corpus = list(set(count.keys()) - set(uncommon))

In [46]:
messy = ['|', '@', '_', '^', 'amp;', 'gt;', 'faq_autotldr_bot/', "](/r", '’', '-', '█', '%', '/r', '*', '°', 'lt;3', \
        '#', '?', '/','‘']
messy2 = ['a','b','c','d','e','f','g','h','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z'] +\
['a.','b.','c.','d.','e.','f.','g.','h.','i.','j.','k.','l.','m.','n.','o.','p.','q.','r.','s.','t.','u.','v.','w.','x.','y.','z.']+\
["'",'.','+','(', ')','*']
messy_common = []
for w in corpus:
    for i in messy:
        if w.find(i) != -1 and w not in messy2 and w.isdigit() != 1:
            messy_common.append(w)
print (len(messy_common))

111


In [47]:
corpus0 = list(set(count.keys()) - set(uncommon) - set(messy_common))
print (len(corpus0))

6044


In [15]:
for k, v in collections.Counter(corpus0).items():
    if k in corpus[1000:2000]:
        print (k)

flirt
discontinu
psychic
cave
classifi
nicht
vegan
engin
incur
leather
teller
raspberri
homebrew
surviv
eastern
purchas
compilebot
thigh
inflict
calc
acquir
techi
wander
pen
alex
stud
credenti
mutil
predat
sole
ascend
downvot
damper
pedo
obsidian
treat
evapor
bottom
fire
dart
genuin
mueller
trustworthi
aquarium
bicycl
extend
arti
mere
spray
5th
facilit
walker
committe
platter
inspect
una
cb
gtx
kkk
bottl
poop
tippr
wrap
wikileak
stare
slam
aesthet
ni
limb
btw
qui
ahl
teammat
hoodi
rite
underdog
m
fring
dong
fluid
sera
over
meetup
arguabl
playoff
previous
format
blossom
berri
magnific
butler
temp
etern
printer
custodi
smurf
point
salti
audienc
patriarch
jc
manual
vive
maintain
rais
m.2
hangout
torqu
swap
peel
dam
ami
cart
consolid
shoe
stalker
bush
origin
js
g
leak
streetwear
repeat
lyon
acronym
hahaha
dw
billionair
ninja
roar
gfe
child
ivanka
dawn
josh
filter
squat
stripe
lucid
wave
attack
terrif
choir
sturdi
el
derek
nvme
materi
counsel
asian
press
damnit
ha
textbox
ele
ghost
few
feed

## Clean depressed posts

In [48]:
words_depr = get_words(text_depr)
count = collections.Counter(words_depr)

The number of unique words: 35085


In [49]:
uncommon = findWordLessThan(count, 5)
common = findWordMoreThan(count, 100)

23488
2768


In [37]:
"""""
# check
for k, v in count.items():
    if k in uncommon:
        print (k, v)

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-37-8c6a887bd622>, line 5)

In [38]:
#uncommon[3000:3100]

In [50]:
corpus = list(set(count.keys()) - set(uncommon))
print (len(corpus))

11597


In [51]:
messy = ['|', '_', '^', 'amp;', 'gt;', '’', '-', '█', '%', '/r', '?', '~', '.', '/', '‘', '#']
messy2 = ['a','b','c','d','e','f','g','h','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z'] + \
['a.','b.','c.','d.','e.','f.','g.','h.','i.','j.','k.','l.','m.','n.','o.','p.','q.','r.','s.','t.','u.','v.','w.','x.','y.','z.']+\
["'",'.','+','=', '―','☆', '(', ')']
messy_common = []
for w in corpus:
    for i in messy:
        if w.find(i) != -1 and w not in messy2 and w.isdigit() != 1:
            messy_common.append(w)
print (len(messy_common))

162


In [52]:
corpus1 = list(set(count.keys()) - set(uncommon) - set(messy_common))
print (len(corpus1))

11437


In [18]:
# top words in depr posts
corpus1_new = []
d = collections.Counter(words_depr)
s1 = [(k, d[k]) for k in sorted(d, key=d.get, reverse=True)]
for k, v in s1[:4000]:
    if k in corpus1:
        corpus1_new.append(k)

# top words in control posts
corpus0_new = []
d = collections.Counter(words_control)
s2 = [(k, d[k]) for k in sorted(d, key=d.get, reverse=True)]
for k, v in s2[:3000]:
    if k in corpus0:
        corpus0_new.append(k)

In [53]:
#corpus = corpus0_new + corpus1_new
corpus = corpus0 + corpus1
print (len(set(corpus)))

12251


## bag-of-words

In [59]:
def get_dic(corpus):
    word_space=defaultdict(int)
    for word in corpus:
        if word not in word_space:
            word_space[word]=len(word_space) 
    print ("The number of unique words in the corpus is " + str(len(word_space)))
    return word_space

def get_sparse_vec(docs, word_space):
    sparse_vecs = []
    for doc in docs:
        # create empty vector
        sparse_vec = np.zeros(len(word_space))
        for word in doc.split():
            if word in word_space:
                sparse_vec[word_space[word]] = 1
        sparse_vecs.append(sparse_vec) 
    print ("The number of samples is " +str(len(sparse_vecs)))
    if len(sparse_vecs[0]) != len(word_space):
        print ("error")
    return sparse_vecs

def get_freq_vec_old(docs, word_space):
    freq_vecs = []
    for doc in docs:
        # create empty vector
        freq_vec = np.zeros(len(word_space))
        for word in doc.split():
            if word in word_space:
                freq_vec[word_space[word]] += 1
        freq_vecs.append(freq_vec) 
    print ("The number of samples is " +str(len(freq_vecs)))
    if len(freq_vecs[0]) != len(word_space):
        print ("error")
    return freq_vecs  


def get_freq_vec(docs, word_space):
    freq_vecs = []
    for doc in docs:
        # create empty vector
        freq_vec = np.zeros(len(word_space))
        for word in doc.split():
            if word in word_space:
                freq_vec[word_space[word]] += 1
        freq_vecs.append(freq_vec) 
    print ("The number of samples is " +str(len(freq_vecs)))
    if len(freq_vecs[0]) != len(word_space):
        print ("error")
    return freq_vecs  

In [33]:
"""""
def get_dic(docs):
    word_space=defaultdict(int)
    for doc in docs:
        for word in doc.split():
            if word not in word_space:
                word_space[word]=len(word_space) 
    return word_space

def get_sparse_vec(doc, word_space):
    # create empty vector
    sparse_vec = np.zeros(len(word_space))
    for word in doc.split():
        try:
            sparse_vec[word_space[word]]=1
        except:
            continue    
    return sparse_vec

In [65]:
"""""
m = ["a", 'b', 'c', 'b','d']
c = get_dic(m)
c

The number of unique words in the corpus is 4


defaultdict(int, {'a': 0, 'b': 1, 'c': 2, 'd': 3})

In [60]:
word_space = get_dic(corpus)

The number of unique words in the corpus is 12251


In [ ]:
#word_space

In [122]:
# save the word_space for later use
outfile = json.dumps(word_space)
f = open("D:/Depression_project/word_space.json","w")
f.write(outfile)
f.close()

# Get binary representation of words

In [20]:
sparse_vecs0 = get_sparse_vec(text_control, word_space)
print (len(sparse_vecs0[0]))
print (sparse_vecs0[0])

MemoryError: 

In [34]:
sparse_vecs1 = get_sparse_vec(text_depr, word_space)
print (sparse_vecs1[0])

MemoryError: 

# Get TF-IDF representation of words

In [123]:
X0 = get_freq_vec(text_control, word_space)
print (len(X0[0]))
print (X0[0])

The number of samples is 50000
12251
[0. 0. 0. ... 0. 0. 0.]


In [124]:
X1 = get_freq_vec(text_depr, word_space)
print (len(X1[0]))
print (X1[0])

The number of samples is 50000
12251
[0. 0. 0. ... 0. 0. 0.]


In [125]:
X = X0 + X1

In [25]:
# tf-idf transform
#transformer = TfidfTransformer(use_idf=True, smooth_idf=True).fit(X)  # true means add one smoothing

In [27]:
# save the tf-idf weights for later use (e.g., predict outcomes of new doc)
#filename = 'D:/Depression_project/tfidf'
#pickle.dump(transformer, open(filename, 'wb'))

# Feature selection for dimensionality reduction

In [4]:
def constr(rangeX=None, rangeY=None):
    if rangeX == None and rangeY == None:
        #X = sparse_vecs0 + sparse_vecs1
        X = X0 + X1
        y0 = [0 for i in range(len(text_control[:rangeX]))]
        y1 = [1 for i in range(len(text_depr[:rangeY]))]
    else:
        #X = sparse_vecs0[:rangeX] + sparse_vecs1[:rangeY]
        X = X0[:rangeX] + X1[:rangeY]
        y0 = [0 for i in range(len(text_control[:rangeX]))]
        y1 = [1 for i in range(len(text_depr[:rangeY]))]            
    
    y = y0 + y1    
    print(len(X))
    print (len(y))
    return X, y

In [127]:
X, y = constr()
print ("The number of features is "+str(len(X[0])))

100000
100000
The number of features is 12251


In [128]:
# select the top k features
selector = SelectKBest(chi2, k=1000)
#selector = SelectPercentile(chi2, percentile=20)
selector.fit(X, y)

SelectKBest(k=1000, score_func=<function chi2 at 0x000001D6AD6692F0>)

In [82]:
# Get parameters for this estimator.
#selector.get_params() 

# Get a mask, or integer index, of the features selected
#selector.get_support()  

array([ True,  True,  True, ..., False, False, False])

In [131]:
# save selected feature to a file for later prediction use
a = list(selector.get_support())
f = open("D:/Depression_project/feature_selection.txt", "w")
for i in a:
    if i == True:
        f.write(str(1)+"\n")
    else: 
        f.write(str(0)+"\n")
f.close()

In [132]:
# fit the selected features to the data
X_subset = selector.fit_transform(X, y)

C:\Users\Bao\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:127: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):


In [133]:
# tf-idf transform
transformer = TfidfTransformer(use_idf=True, smooth_idf=True).fit(X_subset)  # true means add one smoothing

In [134]:
# save the tf-idf weights for later use (e.g., predict outcomes of new doc)
filename = 'D:/Depression_project/tfidf2'
pickle.dump(transformer, open(filename, 'wb'))

In [135]:
X_input = transformer.fit_transform(X_subset)
print ("\nThe number of samples in the training data is "+str(len(X_input.toarray()))+"\n")
print ("\nThe number of features in model is "+str(len(X_input.toarray()[0]))+"\n")

C:\Users\Bao\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [41]:
"""""
X_train = transformer.fit_transform(X_train)
X_train.toarray()

array([[0.09384399, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.20476926, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.12148246, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.28487243, 0.05853809, 0.18069294, ..., 0.        , 0.        ,
        0.        ],
       [0.32111857, 0.17321395, 0.07638137, ..., 0.        , 0.        ,
        0.        ],
       [0.15055485, 0.12374934, 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [95]:
"""""
a = list(selector.get_support())
f = open("feature_selection.txt" , "w")
for i in a:
    if i == True:
        f.write(str(1)+"\n")
    else: 
        f.write(str(0)+"\n")
f.close()

# Train a classifier

Now that we have our features, we can train a classifier to try to predict the category of a post. Let’s start with svm classifier

In [52]:
"""""
clf = Pipeline([
  ('feature_selection', SelectFromModel(linear_model.LogisticRegression(C=0.1))),
  ('classification', svm.SVC(kernel='linear', probability=True))
])
clf.fit(X_train, y_train)
print ("\nDone fitting classifier on training data...\n")


Done fitting classifier on training data...



C:\Users\Bao\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:127: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):


In [ ]:
svmModel = svm.SVC(kernel='linear', probability=True)
svmModel.fit(X_input, y)
print ("\nDone fitting classifier on training data...\n")

In [ ]:
# save the classifier to disk
filename = 'D:/Depression_project/svmModel2.sav'
pickle.dump(svmModel, open(filename, 'wb'))
print ("\nModel saved to disk.\n")

In [90]:
predicted = cross_validation.cross_val_predict(svmModel, X_input, y, cv=10)
print ("*"*20)
print ("\t accuracy_score\t", metrics.accuracy_score(y, predicted))
print ("*"*20)
print ("precision_score\t", metrics.precision_score(y, predicted))
print ("recall_score\t", metrics.recall_score(y, predicted))
print ("\nclassification_report:\n\n", metrics.classification_report(y, predicted))
print ("\nconfusion_matrix:\n\n", metrics.confusion_matrix(y, predicted))

********************
	 accuracy_score	 0.815
********************
precision_score	 0.8620689655172413
recall_score	 0.75

classification_report:

              precision    recall  f1-score   support

          0       0.78      0.88      0.83       100
          1       0.86      0.75      0.80       100

avg / total       0.82      0.81      0.81       200


confusion_matrix:

 [[88 12]
 [25 75]]


Let us try Logistic regression

In [ ]:
lrModel = linear_model.LogisticRegression(random_state=0)
lrModel.fit(X_input, y)
print ("\nDone fitting classifier on training data...\n")

In [ ]:
# save the classifier to disk
filename = 'D:/Depression_project/lrModel.sav'
pickle.dump(lrModel, open(filename, 'wb'))
print ("\nModel saved to disk.\n")

In [ ]:
predicted = cross_validation.cross_val_predict(lrModel, X_input, y, cv=10)
print ("*"*20)
print ("\t accuracy_score\t", metrics.accuracy_score(y, predicted))
print ("*"*20)
print ("precision_score\t", metrics.precision_score(y, predicted))
print ("recall_score\t", metrics.recall_score(y, predicted))
print ("\nclassification_report:\n\n", metrics.classification_report(y, predicted))
print ("\nconfusion_matrix:\n\n", metrics.confusion_matrix(y, predicted))

# Predict outcome of new document

## Load and clean twitter data

In [5]:
# specify dir
Dir = 'D:/Depression_project/data/twitter'

In [6]:
texts = []
ids = []
#with open(Dir+'/'+'tweets_cleaned.txt',"r", errors = 'ignore') as f:
with open(Dir+'/'+'tweetsEng_cleaned_2.txt',"r", errors = 'ignore') as f:
    data = f.readlines()
texts = [x.strip().split('#^~^#')[1] for x in data] 
ids = [x.strip().split('#^~^#')[0] for x in data] 
len(texts)

171514

In [7]:
words = get_words(texts)
count = collections.Counter(words)

The number of unique words: 131316


In [8]:
uncommon = findWordLessThan(count, 10)
common = findWordMoreThan(count, 100)

120912
1955


In [9]:
corpus = list(set(count.keys()) - set(uncommon))
print (len(corpus))

10404


In [10]:
messy_exact = ['a','b','c','d','e','f','g','h','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']+['a.','b.',\
'c.','d.','e.','f.','g.','h.','i.','j.','k.','l.','m.','n.','o.','p.','q.','r.','s.','t.','u.','v.','w.','x.','y.','z.']
messy = messy_exact + ["'",'.','+','=', ':','rt']
messy2 = ['#', '’', '@', 'rt', '_',  'http', '%', 'x', '…', '|', '/', '▔', 'amp', '┏', '★', '═', 'gt;',';&','lt;','】','↙'\
'！', '+','╝','╔', '╚', '━', '║', '『','╮','╭', '°', '―', '█','◆', '┃','「','-','╱', '┓','▏', '╗', '─',\
'┛','╲','」','“','┻','┗',',','?','×','#']
messy_common = []
for w in corpus:
    for i in messy2:
        if w.find(i) != -1 and w not in messy and w.isdigit() != 1:
            messy_common.append(w)
print (len(messy_common))

2445


In [11]:
corpus = list(set(count.keys()) - set(uncommon) - set(messy_common))
print (len(corpus))

8324


## Get features using tf-idf representation

In [12]:
# load word_space 
with open("D:/Depression_project/word_space2.json", mode='r') as file:  
    word_space = json.load(file)   # json.load reads whole json file, json.loads reads string only
print (len(word_space))

12262


In [13]:
freq_vecs = get_freq_vec(texts, word_space)
print (len(freq_vecs[0]))
print (freq_vecs[0])

The number of samples is 171514
12262
[0. 0. 0. ... 0. 0. 0.]


In [14]:
# only include the selected features
newX = []
with open("D:/Depression_project/feature_selection.txt", "r") as f:
    reader = f.readlines()
    for x in freq_vecs[:2000]: 
        count = 0
        x_subset = []
        for i in reader:
            if i.strip() == "1":
                x_subset.append(freq_vecs[count])
            count += 1
        newX.append(x_subset)
print (len(newX))

2000


In [27]:
# only include the selected features
binary = []
with open("D:/Depression_project/feature_selection.txt", "r") as f:
    reader = f.readlines()
    for i in reader:
        if i.strip() == "1":
            binary.append(1)
        else:
            binary.append(0)
#newX = np.array(freq_vecs) * np.array(binary)            
#print (len(newX))
print (len(binary))

12251


In [25]:
"""""
# only include the selected features
newX = []
for i in selector.get_support():
    if i == True:
        newX.append(freq_vecs[i])
print (len(newX))

In [15]:
# load the tfidf result
filename = 'D:/Depression_project/tfidf2'
transformer = pickle.load(open(filename, 'rb'))

In [68]:
X_new = transformer.transform(newX)

ValueError: unrecognized csr_matrix constructor usage

In [17]:
# load the classifier
filename = 'D:/Depression_project/svmModel_tfidf2.sav'
svmModel = pickle.load(open(filename, 'rb'))

In [26]:
svmModel.coef_.toarray()

[array([ 5.53827806e-01,  8.60170715e-01,  1.02946020e+00,  1.33578532e+00,
        -3.90472004e-02,  1.03252033e+00,  1.66634532e+00,  6.92771160e-01,
        -6.21543796e-01,  1.48108084e+00, -5.73898778e-01,  6.22504293e-01,
        -2.83400244e-01,  3.34224049e-01,  9.44553837e-01,  1.47172234e+00,
         1.52986850e+00,  8.03619285e-01,  5.39765086e-01, -2.67227283e-02,
         3.42955469e-01,  1.37520748e+00,  2.29407391e+00,  3.00380250e-02,
         3.70507545e-01, -1.36799078e-01, -9.76038941e-02,  3.47119871e-01,
         7.46828775e-01,  1.16175737e+00,  4.65332224e-01, -9.05059946e-01,
         1.80688434e+00,  1.73963948e+00,  7.78027282e-01,  7.51764825e-01,
        -2.28854264e-01,  5.70453892e-01,  7.29928413e-01,  1.03956703e+00,
        -5.18478970e-01,  5.34139297e-01,  9.44197011e-01,  2.05060722e+00,
         1.90026631e+00,  1.43922838e+00,  1.41806093e+00, -2.01389949e-01,
         3.72448477e+00,  2.62054448e-01,  1.59930941e-01,  8.42730128e-01,
         1.5

In [71]:
feature_index = []
count = 0
for i in binary:
    if i == 1:
        feature_index.append(count)
    count += 1
print (len(feature_index))
print (feature_index[:10])

1000
[8, 21, 27, 39, 40, 44, 45, 47, 50, 59]


In [72]:
feature_show = []
for i in feature_index:
    for k, v in word_space.items():
        if i == v:
            feature_show.append(k)
print (len(feature_show))
print (feature_show[:10])        

1000
['misread', 'sugar', 'veget', 'well', 'dri', 'empir', 'note', 'allow', 'difficult', 'suffic']


In [73]:
print (feature_show) 

['misread', 'sugar', 'veget', 'well', 'dri', 'empir', 'note', 'allow', 'difficult', 'suffic', 'whi', 'potter', 'frost', 'show', 'instal', 'territori', 'injustic', 'reduc', 'cd', 'splash', 'histori', 'jay', 'throat', 'stink', 'bolt', 'hardwar', 'bargain', 'forest', 'comput', 'urban', 'deliv', 'steve', 'singer', 'vote', 'rad', 'underwear', 'turtl', 'rememb', 'stay', 'har', 'vm', 'therapi', 'upvot', 'closer', 'shame', 'australian', 'chore', 'final', 'rampant', 'short', 'recognit', 'lisa', 'marri', 'l.', 'off', 'temp', 'pepe', 'sister', 'pedal', 'cooper', 'l', 'fox', 'treati', 'elsewher', 'biscuit', 'draw', 'be', 'dinner', 'sweater', 'freezer', 'discord', 'medicin', 'cardio', 'guidelin', 'pg', 'sarcasm', 'britain', 'nice', 'quick', 'acronym', 'element', 'border', 'franchis', 'glow', 'intim', 'pixel', 'nazi', 'friday', 'vietnam', 'mad', 'subscript', 'tini', 'everyth', 'lanc', 'standard', 'produc', 'modern', 'surprisingli', 'messag', 'wii', 'gamma', 'approach', 'ms', 'alloc', 'roll', 'taylor

In [34]:
#predY = svmModel.predict(X_new)

In [35]:
predP = svmModel.predict_proba(X_new)

In [38]:
f = open(Dir+'/'+'predictedProbResults_2.txt', "w")
count = 0
for p in predP:
    f.write(str(ids[count])+' '+str(round(p[1], 2))+'\n')
    count += 1 
f.close()